# Данный нотбук делает следующее:

*   Обучение Yolov8 на кастомном датасете с гугл диска/robflow
*   Экспорт модели в кастомный ONNX через репозиторий yolov8 от RKNN
*   Конвертация ONNX модели в RKNN



### Настройка окружений

In [ ]:
!apt install python3.10-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,474 kB of archives.
After this operation, 2,885 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.2 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.6 [5,722 B]
Fetched 2,474 kB in 1s (2,757 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 123622 files and directories currently installed.)
Pr

In [ ]:
!pip3 install ultralytics -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 20.3 MB/s eta 0:00:00


### Копирование датасета с гугл диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Укажите путь к датасету, экспортированному с Roboflow в zip архиве

In [ ]:
%cd /content
!cp /content/drive/MyDrive/NozzleData/nozzle_v.0.2.1.zip .
!unzip nozzle_v.0.2.1.zip -d nozzle_data

In [ ]:
!rm nozzle_v.0.2.1.zip

**В data.yaml указать абсолютные пути к директориям с картинками**

In [ ]:
%cd /content

/content


In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'

### Обучение

В переменной **SIZE** указывается размер изображений для обучения

In [ ]:
from ultralytics import YOLO

SIZE = 640
model = YOLO('yolov8n.pt')
model.train(data='/content/nozzle_data/data.yaml', epochs=25, device=0, batch=128, imgsz=SIZE, augment=False)

### Некоторые тесты (необязательно)

In [ ]:
metrics = model.val(split="test", plots=True)

Ultralytics 8.3.18 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/nozzle_data/test/labels.cache... 41 images, 1 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


                   all         41         41      0.931          1      0.989      0.932
                 esp01         20         20      0.997          1      0.995      0.906
           nozzle-8hmp         21         21      0.865          1      0.983      0.959
Speed: 0.3ms preprocess, 5.3ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
!yolo detect val model=/content/runs/detect/train/weights/best.pt data=/content/nozzle_data/data.yaml

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 754, in entrypoint
    check_dict_alignment(full_args_dict, {a: ""})
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 436, in check_dict_alignment
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'test' is not a valid YOLO argument. Similar arguments are i.e. ['detect'].

    Arguments received: ['yolo', 'detect', 'test', 'model=/content/runs/detect/train/weights/best.pt', 'data=/content/nozzle_data/data.yaml']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'obb', 'classify', 'segment', 'pose', 'detect'}
                MODE (required) is one of {'train', 'track', 'val', 'benchmark', 'export', 'predict'}
                ARGS (optional) are any number of custom 'arg=

### Копирование результатов на гугл диск

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Вся папка обучения - графики, метрики и веса
!cp -r /content/runs/detect/train /content/drive/MyDrive

### Конвертация в ONNX

#### Установка

In [ ]:
!git clone https://github.com/airockchip/ultralytics_yolov8
%cd ultralytics_yolov8
!python3 -m venv env
!source env/bin/activate
!pip3 uninstall ultralytics -y

In [ ]:
!pip3 install onnx onnxruntime

#### Экспорт

Не забудьте поменять путь к модели в файле ultralytics/cfg/default.yaml

In [ ]:
!PYTHONPATH=./ python ./ultralytics/engine/exporter.py

In [ ]:
!cp /content/runs/detect/train/weights/best.onnx /content/drive/MyDrive/nozzle_esp_320_px_correct.onnx

### Экспорт в RKNN

#### Установка

In [ ]:
%cd /content
!python3 -m venv env
!source env/bin/activate
!wget https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/rknn_toolkit2-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip3 install rknn_toolkit2-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [ ]:
import rknn

In [ ]:
!git clone https://github.com/airockchip/rknn_model_zoo
%cd rknn_model_zoo/examples/yolov8

In [ ]:
import os
import cv2
from tqdm import tqdm

# Если нужно экспортировать модель с другими размерами изображений (не как в датасете), иначе None
RESIZE_TO_IMGSZ = None # или None, если не нужно ресайзить

# TODO желательно распараллелить

IMG_PATH = "/content/nozzle_data/train/images"
files = os.listdir(IMG_PATH)

with open("/content/data_subset.txt", "w") as fd:
  for i in tqdm(files):
    print(os.path.join(IMG_PATH, i), file=fd)
    if RESIZE_TO_IMGSZ:
      img = cv2.imread(os.path.join(IMG_PATH, i))
      img = cv2.resize(img, (RESIZE_TO_IMGSZ, RESIZE_TO_IMGSZ))
      cv2.imwrite(os.path.join(IMG_PATH, i), img)

100%|██████████| 864/864 [00:00<00:00, 316828.00it/s]


In [ ]:
%cd python

/content/rknn_model_zoo/examples/yolov8/python


Укажите путь /content/data_subset.txt в файле convert.py

In [ ]:
!python3 convert.py /content/runs/detect/train/weights/best.onnx rv1103 i8

In [ ]:
!cp ../model/yolov8.rknn /content/drive/MyDrive/nozzle_esp_320_real.rknn